In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import re

def get_average_watch_price(brand_name, driver, is_fallback=False):
    url = f"https://www.chrono24.com/search/index.htm?dosearch=true&query={brand_name.replace(' ', '+')}"
    driver.get(url)
    time.sleep(0.7)

    try:
        # Accept cookies if needed
        try:
            cookies_button = driver.find_element(By.CSS_SELECTOR, "button[class^='js-cookie-accept-required']")
            cookies_button.click()
            time.sleep(2)
        except: pass
        
        try:
            close_button = driver.find_element(By.CSS_SELECTOR, ".btn.btn-secondary.flex-equal.w-100-sm.m-r-sm-5.js-close-modal")
            close_button.click()
            time.sleep(1)
        except: pass    

        price_elements = driver.find_elements(By.CSS_SELECTOR, ".js-article-item.article-item.block-item.rcard .text-bold")
        prices = []

        for element in price_elements:
            try:
                price_text = element.text.strip().replace('$', '').replace(',', '').strip()
                if price_text.replace('.', '').isdigit():
                    price = float(price_text)
                    if price > 0:
                        prices.append(price)
            except:
                continue

        if prices:
            avg_price = sum(prices[:5]) / len(prices[:5])
            return round(avg_price, 2)

        # fallback: try searching with just the first word if multi-word
        if not is_fallback and ' ' in brand_name:
            fallback_term = brand_name.split()[0]
            print(f"❗ No prices found for '{brand_name}', retrying with fallback term '{fallback_term}'...")
            return get_average_watch_price(fallback_term, driver, is_fallback=True)

        return None

    except Exception as e:
        print(f"⚠️ Error fetching price for {brand_name}: {e}")
        return None


def classify_brand_tier(avg_price):
    if avg_price is None:
        return "unknown"
    elif avg_price >= 3000:
        return "luxury"
    elif avg_price >= 1000:
        return "mid-range"
    else:
        return "budget"


In [21]:
def scrape_brands_price_index(brands):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    with open('watch_brand_prices.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Brand', 'Average Price (€)', 'Tier'])

        for brand in brands:
            print(f"Processing: {brand}")
            avg_price = get_average_watch_price(brand, driver)
            tier = classify_brand_tier(avg_price)
            print(f"{brand}: {avg_price} → {tier}")
            writer.writerow([brand, avg_price, tier])

    driver.quit()

# Example use


In [3]:
# get brand names from excel
import pandas as pd



In [33]:
file_path = "chrono24_reviews.xlsx"

df = pd.read_excel(file_path)


In [34]:
df['brand']

0                                    Oris Divers Sixty-Five
1         \n               TAG Heuer Carrera Cal.  5 aut...
2                      Tissot T-Sport Quickster Chronograph
3                                Tissot PRS 516 Chronograph
4                  Frederique Constant Vintage Rally Healey
                                ...                        
175963          \n               Alpina vintage manual wind
175964                    \n               Elysee Automatic
175965              \n               Oris Date, manual wind
175966    \n               KASPER chronograph, manual wi...
175967                                           Zenith ...
Name: brand, Length: 175968, dtype: object

In [12]:
import sys
import os

# Add current working directory to path (since brand_processing is here)
sys.path.append(os.path.abspath("."))

# Now import the function
from brand_processing.process_brands import process_brand_for_search

# Get the brand names from the dataframe
brand_names = df['brand'].tolist()
# create a dict with the original and the processed brand names
brand_dict = {}
for brand in brand_names:
    processed_brand = process_brand_for_search(brand)
    brand_dict[brand] = processed_brand

In [35]:
# leave only 2 first words f=in processed brands
brand_dict = {k: ' '.join(v.split()[:2]) for k, v in brand_dict.items()}

In [36]:
brand_dict

{'Oris Divers Sixty-Five': 'oris divers',
 '\n               TAG Heuer Carrera Cal.  5 automatic 36mm': 'tag heuer',
 'Tissot T-Sport Quickster Chronograph': 'tissot t',
 'Tissot PRS 516 Chronograph': 'tissot prs',
 'Frederique Constant Vintage Rally Healey': 'frederique constant',
 'Hamilton Jazzmaster GMT': 'hamilton jazzmaster',
 'Zenith PILOT Type 20 Chronograph Extra Special': 'zenith pilot',
 '\n               Union Glashütte 1893 sub second': 'union glashutte',
 '\n               Alpina Startimer Pilot automatic - limited': 'alpina startimer',
 'Oris Aquis Date 43,5 mm': 'oris aquis',
 'Montblanc ...': 'montblanc',
 'Oris ...': 'oris',
 '\n               Day, this year ...\n                  ': 'day this',
 'Alpina ...': 'alpina',
 'Frederique Constant ...': 'frederique constant',
 '\n               Alpina ...\n                  ': 'alpina',
 'Bomberg ...': 'bomberg',
 'Mido ...': 'mido',
 'Bulova ...': 'bulova',
 '\n               Bulova ...\n                  ': 'bulova',
 '\n

In [37]:
# add the processed brand names to the dataframe
df['processed_brand'] = df['brand'].map(brand_dict)

In [16]:
df.head()

,brand,buyer_comment,buyer_country_code,review_date,review_url,reviews month,reviews year,has_comment,seller_uid,buyer_country,processed_brand
0,Oris Divers Sixty-Five,Everything great.,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,oris divers
1,\n TAG Heuer Carrera Cal. 5 aut...,Super fast delivery. Goods are perfect and co...,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,tag heuer
2,Tissot T-Sport Quickster Chronograph,"Exactly as described, super fast delivery, I w...",DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,tissot t
3,Tissot PRS 516 Chronograph,-,DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,Germany,tissot prs
4,Frederique Constant Vintage Rally Healey,-,FR,06/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,France,frederique constant


In [24]:

brands = df['processed_brand'].tolist()
# remove duplicates
brands = list(set(brands))
scrape_brands_price_index(brands)



Processing: 
: None → unknown
Processing: maserati r8853100022
maserati r8853100022: 239.33 → budget
Processing: charriol parisi
charriol parisi: 1165.0 → mid-range
Processing: prexa 18k
❗ No prices found for 'prexa 18k', retrying with fallback term 'prexa'...
prexa 18k: 391.0 → budget
Processing: hermes hour
hermes hour: 7823.33 → luxury
Processing: english deadly
❗ No prices found for 'english deadly', retrying with fallback term 'english'...
english deadly: 47135.0 → luxury
Processing: thermidor new
thermidor new: 280.8 → budget
Processing: vanroy faceted
❗ No prices found for 'vanroy faceted', retrying with fallback term 'vanroy'...
vanroy faceted: 1071.0 → mid-range
Processing: lyndamar
lyndamar: None → unknown
Processing: rolex pearlmaster
rolex pearlmaster: 107210.0 → luxury
Processing: jacques
jacques: 1072.4 → mid-range
Processing: longines ultrachron
longines ultrachron: 1517.4 → mid-range
Processing: strap compatible
strap compatible: 1000.0 → mid-range
Processing: citizen b

In [38]:
# add the price and tier to the dataframe
prices_df = pd.read_csv('Product_Segments.csv')
prices_df

,Brand,Average Price (€),Tier
0,NaN,NaN,unknown
1,maserati r8853100022,239.33,budget
2,charriol parisi,1165.00,mid-range
3,prexa 18k,391.00,budget
4,hermes hour,7823.33,luxury
...,...,...,...
3286,emka sa,1854.00,mid-range
3287,corum rubber,4650.20,luxury
3288,hermes heure,8384.00,luxury
3289,tabbah,1630.80,mid-range


In [39]:
# map the prices and tiers to the original dataframe
df = df.merge(prices_df, left_on='processed_brand', right_on='Brand', how='left')

In [40]:
df

,brand,buyer_comment,buyer_country_code,review_date,review_url,reviews month,reviews year,has_comment,seller_uid,buyer_country,processed_brand,Brand,Average Price (€),Tier
0,Oris Divers Sixty-Five,Everything great.,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,oris divers,oris divers,1303.6,mid-range
1,\n TAG Heuer Carrera Cal. 5 aut...,Super fast delivery. Goods are perfect and co...,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,tag heuer,tag heuer,1019.6,mid-range
2,Tissot T-Sport Quickster Chronograph,"Exactly as described, super fast delivery, I w...",DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany,tissot t,tissot t,335.0,budget
3,Tissot PRS 516 Chronograph,-,DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,Germany,tissot prs,tissot prs,460.0,budget
4,Frederique Constant Vintage Rally Healey,-,FR,06/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,France,frederique constant,frederique constant,1333.6,mid-range
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175963,\n Alpina vintage manual wind,I am very satisfied with my purchase. The watc...,US,01/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,25100,USA,alpina manual,alpina manual,361.5,budget
175964,\n Elysee Automatic,-,RO,31/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,0,25100,Romania,elysee,elysee,248.0,budget
175965,"\n Oris Date, manual wind",Top ! Everything great !,DE,30/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,25100,Germany,oris date,oris date,1084.8,mid-range
175966,"\n KASPER chronograph, manual wi...","Polite, competent and reliable staff. The obj...",IT,19/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,25100,Italy,kasper manual,kasper manual,273.0,budget


In [41]:
# export the dataframe to excel
df.to_excel('chrono24_reviews_with_prices.xlsx', index=False)